In [18]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from math import sqrt
import scipy as sp

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


from sksurv.util import Surv
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from xgbse.converters import (convert_to_structured, convert_data_to_xgb_format)
import xgboost as xgb
from xgbse.metrics import concordance_index


# Our very own survLime!
import survLime
from survLime.datasets.load_datasets import Loader 
from survLime import survlime_explainer

dataset = 'udca'
loader = Loader(dataset_name=dataset)
x,events, times = loader.load_data()


train, val, test = loader.preprocess_datasets(x, events, times, random_seed=0, standarize=True)
# conver labels
train[1] = pd.DataFrame(train[1])
val[1] = pd.DataFrame(val[1])
test[1] = pd.DataFrame(test[1])

y_train = convert_to_structured(train[1]['time'], train[1]['event'])
y_val = convert_to_structured(val[1]['time'], val[1]['event'])
y_test = convert_to_structured(test[1]['time'], test[1]['event'])

# and data
dtrain = convert_data_to_xgb_format(train[0], y_train, 'survival:aft')
dval = convert_data_to_xgb_format(val[0], y_val, 'survival:aft')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/carlos.hernandez/dl/lib/python3.8/site-packages/pandas/core/series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [29]:
# Instantiate some hyperparams
PARAMS_XGB_AFT = {
    'objective': 'survival:aft', # <- we could also use survival:cox
    'eval_metric': 'aft-nloglik',
}


bst = xgb.train(
        PARAMS_XGB_AFT,
        dtrain,
        num_boost_round=10,
        early_stopping_rounds=10,
        evals=[(dval, 'val')],
        verbose_eval=0
    )

In [21]:
#dtest = convert_data_to_xgb_format(test[0], y_test, 'survival:aft')
dtest = xgb.DMatrix(test[0], label=np.zeros(len(test[0])))

preds = bst.predict(dtest)
c_index = concordance_index(y_test, -preds, risk_strategy='precomputed')
print(f'Hooray we got {round(c_index,3)} of concordance index')

Hooray we got 0.567 of concordance index


### Now its SurvTIME! (hehe)

In [31]:

preds

array([2742.442  , 2742.442  , 3338.7915 , 2742.442  , 2742.442  ,
       2280.9048 ,  795.93365, 2742.442  , 5341.677  , 3338.7915 ,
       2742.442  , 2742.442  , 2742.442  ,  939.4978 , 7919.159  ,
       2742.442  ,  170.53593, 1256.4805 , 1164.961  , 2742.442  ,
       2742.442  ,  879.89435, 2742.442  , 2742.442  , 2742.442  ,
       2742.442  ], dtype=float32)